In [1]:
# The text classification for tweets --- COMP90049  Assignment 3 

In [2]:
# import components
import numpy as np
import pandas as pd

import os



In [3]:
# 1. load dataset
# data 
data_train_full=pd.read_csv('data/train_full.csv')
data_train_count=pd.read_csv('data/train_count.csv')
data_train_glove300=pd.read_csv('data/train_glove300.csv')
data_train_tfidf=pd.read_csv('data/train_tfidf.csv')

data_test_full=pd.read_csv('data/test_full.csv')
data_test_count=pd.read_csv('data/test_count.csv')
data_test_glove300=pd.read_csv('data/test_glove300.csv')
data_test_tfidf=pd.read_csv('data/test_tfidf.csv')

data_validation_full=pd.read_csv('data/dev_full.csv')
data_validation_count=pd.read_csv('data/dev_count.csv')
data_validation_glove300=pd.read_csv('data/dev_glove300.csv')
data_validation_tfidf=pd.read_csv('data/dev_tfidf.csv')




In [4]:
data_train_full.head()

,region,user,tweet
0,NORTHEAST,USER_6197f95d,Watching LOST
1,NORTHEAST,USER_6197f95d,@USER_89a3500b i did
2,NORTHEAST,USER_6197f95d,"Maneuver so that I can put my team on, hopeful..."
3,NORTHEAST,USER_6197f95d,Darko was eating hamburgers in the locker room...
4,NORTHEAST,USER_6197f95d,Girl pack ya bags i'm bout to take you on a ride!


In [5]:
data_test_full.head()

,region,user,tweet
0,?,USER_e34edf78,"#nowplaying trey songz - does she know...""She ..."
1,?,USER_e34edf78,#chancesare #unotfromthehoodif you walk in the...
2,?,USER_e34edf78,#nowplayin 50 cent -think about me.......shit ...
3,?,USER_e34edf78,@USER_c765e9cb me 2!!!!!!!!!lol
4,?,USER_e34edf78,@USER_78e252f4 whudd up?


In [6]:
data_validation_full.head()

,region,user,tweet
0,NORTHEAST,USER_e96c1548,@USER_1c9511b6 I'm cool just getting excited a...
1,NORTHEAST,USER_e96c1548,@USER_69c243a1 Hey now you can say Shoot I got...
2,NORTHEAST,USER_e96c1548,@USER_8d0f33bf hey did you here that wbs is back
3,NORTHEAST,USER_e96c1548,@USER_93fdce74 @USER_6a96a5de @USER_c3df93e6 @...
4,NORTHEAST,USER_e96c1548,@USER_20d86ee6 I'm doing trying to do this die...


In [7]:
print(data_train_full['region'].unique())
print(data_validation_full['region'].unique())

['NORTHEAST' 'MIDWEST' 'SOUTH' 'WEST']
['NORTHEAST' 'SOUTH' 'WEST' 'MIDWEST']


In [8]:
data_train_full.isnull().sum()


region    0
user      0
tweet     0
dtype: int64

In [9]:
data_test_full.isnull().sum()

region    0
user      0
tweet     0
dtype: int64

In [10]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data_train_full['region']=le.fit_transform(data_train_full['region'])
data_validation_full['region']=le.fit_transform(data_validation_full['region'])

y_train_1hot = pd.get_dummies(data_train_full['region'])
y_validation_1hot = pd.get_dummies(data_validation_full['region'])
data_train_full.head()

,region,user,tweet
0,1,USER_6197f95d,Watching LOST
1,1,USER_6197f95d,@USER_89a3500b i did
2,1,USER_6197f95d,"Maneuver so that I can put my team on, hopeful..."
3,1,USER_6197f95d,Darko was eating hamburgers in the locker room...
4,1,USER_6197f95d,Girl pack ya bags i'm bout to take you on a ride!


In [11]:
y_train_1hot

,0,1,2,3
0,0,1,0,0
1,0,1,0,0
2,0,1,0,0
3,0,1,0,0
4,0,1,0,0
...,...,...,...,...
133790,0,0,1,0
133791,0,0,1,0
133792,0,0,1,0
133793,0,0,1,0


In [12]:
y_validation_1hot

,0,1,2,3
0,0,1,0,0
1,0,1,0,0
2,0,1,0,0
3,0,1,0,0
4,0,1,0,0
...,...,...,...,...
11470,0,1,0,0
11471,0,1,0,0
11472,0,1,0,0
11473,0,1,0,0


In [13]:
print(le.inverse_transform([0,1,2,3]))

['MIDWEST' 'NORTHEAST' 'SOUTH' 'WEST']


In [14]:
data_validation_full.head()

,region,user,tweet
0,1,USER_e96c1548,@USER_1c9511b6 I'm cool just getting excited a...
1,1,USER_e96c1548,@USER_69c243a1 Hey now you can say Shoot I got...
2,1,USER_e96c1548,@USER_8d0f33bf hey did you here that wbs is back
3,1,USER_e96c1548,@USER_93fdce74 @USER_6a96a5de @USER_c3df93e6 @...
4,1,USER_e96c1548,@USER_20d86ee6 I'm doing trying to do this die...


In [15]:
# label numbers
data_train_full['region'].value_counts()

1    52582
2    49901
3    16228
0    15084
Name: region, dtype: int64

In [16]:
# 2. provided dataset explore
len(data_train_full['user'].unique())
len(data_validation_full['user'].unique())




300

In [17]:
# 3. raw data cleaning  
train_x, train_y = data_train_full.iloc[:, [1,2]].copy(),data_train_full['region'].copy()
test_x = data_test_full['tweet'].copy()
validation_x,validation_y = data_validation_full.iloc[:, [1,2]].copy(),data_validation_full['region'].copy()
train_x.head()

,user,tweet
0,USER_6197f95d,Watching LOST
1,USER_6197f95d,@USER_89a3500b i did
2,USER_6197f95d,"Maneuver so that I can put my team on, hopeful..."
3,USER_6197f95d,Darko was eating hamburgers in the locker room...
4,USER_6197f95d,Girl pack ya bags i'm bout to take you on a ride!


In [18]:
import re
import string
#from nltk.corpus import stopwords

#stop_word = stopwords.words('english')

def textcleaning(text):
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    
    # remove @
    text=re.sub(r'@\w+', ' ', text)
    # remove url
    text=re.sub(r'http\S+',' ', text)
    # remove # 
    text=re.sub(r'#\w+', ' ', text)
    text=re.sub(r'<.*?>', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    text=re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    #text = ' '.join([word.lower() for word in text if word.lower() not in stop_word])
    return text

In [19]:
train_x_clean = train_x['tweet'].apply(textcleaning)
validation_x_clean = validation_x['tweet'].apply(textcleaning)

train_x_clean.head()

0                                        Watching LOST
1                                                i did
2    Maneuver so that I can put my team on  hopeful...
3    Darko was eating hamburgers in the locker room...
4    Girl pack ya bags i am bout to take you on a r...
Name: tweet, dtype: object

In [20]:
train_y.value_counts(normalize= True)

1    0.393004
2    0.372966
3    0.121290
0    0.112740
Name: region, dtype: float64

In [21]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


tokenizer=Tokenizer(num_words=10000,oov_token='</OOV>')
tokenizer.fit_on_texts(train_x_clean)

vocab_size = len(tokenizer.word_index) + 1
print('vocabulary size:{}'.format(vocab_size))
max_len = round(sum(train_x_clean.apply(len))/len(train_x_clean))
print('max length of input:{}'.format(max_len))

train_x_sequences = tokenizer.texts_to_sequences(train_x_clean)

train_x_padded = pad_sequences(train_x_sequences, max_len, padding='post')

validation_x_sequences=tokenizer.texts_to_sequences(validation_x)

validation_x_padded = pad_sequences(validation_x_sequences, max_len, padding='post')

print('example x:{}'.format(train_x_padded[4]))

vocabulary size:57033
max length of input:57
example x:[ 123 1819   76 3182    2   18  100    5  144   13   19    6  711    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0]


In [22]:
train_x_padded.shape[1]

57

In [23]:
from tensorflow.keras import layers, Sequential
from tensorflow.keras.optimizers import Adam


model_1 = Sequential([
    layers.Embedding(10000,128, input_length=max_len),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(8,activation='relu'),
    tf.keras.layers.Dense(4,activation='softmax')
])
model_1.summary()
model_1.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history_1=model_1.fit(train_x_padded,train_y,epochs=4,validation_data=(validation_x_padded,validation_y))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 57, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 8)                 1032      
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 36        
Total params: 1,281,068
Trainable params: 1,281,068
Non-trainable params: 0
_________________________________________________________________
Epoch 1/4
 307/4182 [=>............................] - ETA: 48s - loss: 1.3560 - accuracy: 0.3843- ETA: 

KeyboardInterrupt: 

In [ ]:
model_2 = Sequential([
    layers.Embedding(10000,128, input_length=max_len),
    layers.LSTM(512),
    layers.Dense(256,activation='relu'),
    layers.Dense(128,activation='relu'),
    layers.Dense(32,activation='relu'),
    layers.Dense(4,activation='softmax')
])
model_2.summary()
model_2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history_2=model_2.fit(train_x_padded,train_y,epochs=4,validation_data=(validation_x_padded,validation_y))

In [ ]:
model_2 = Sequential([
    layers.Embedding(10000,128, input_length=max_len),
    
    layers.Dense(256,activation='relu'),
    layers.Dense(128,activation='relu'),
    layers.Dense(32,activation='relu'),
    layers.Dense(4,activation='softmax')
])